In [6]:
# ✅ 基础导入
import torch
from torchvision.transforms import Resize
from torchvision.transforms.functional import InterpolationMode

# ✅ 项目路径导入
import sys
sys.path.append(".")  # 确保当前路径下可以导入本地模块

# ✅ 从原文件复用
from generate_t2i_sr import get_args
from dit.model import get_model, DiffusionEngine

# ✅ 获取默认 CLI 参数并手动设置部分值（覆盖命令行）
args = get_args()
args.network = 'ckpt/mp_rank_00_model_states.pt'
args.inference_type = 'full'
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ✅ 构建模型并加载权重
net = get_model(args, DiffusionEngine).to(args.device).eval()
ckpt = torch.load(args.network, map_location=args.device)
net.load_state_dict(ckpt, strict=False)
print("✅ 模型构建 & 权重加载完成")


ImportError: cannot import name 'get_model' from 'dit.model' (/root/Inf-DiT-with-TaylorSeer/dit/model.py)

In [ ]:
# 🧪 构造模拟输入：低分辨率图像 + 上采样图 + 噪声图
B, C, H, W = 1, 3, 64, 64
scale = 4
HR_H, HR_W = H * scale, W * scale
device = args.device

lr_img = torch.randn(B, C, H, W).to(torch.float32).to(device)
upsampled_img = Resize((HR_H, HR_W), interpolation=InterpolationMode.BICUBIC)(lr_img)
noisy_img = torch.randn(B, C, HR_H, HR_W).to(torch.float32).to(device)
concat_img = torch.cat([upsampled_img, noisy_img], dim=1)  # [B, 6, HR_H, HR_W]
sigmas = torch.ones(B).to(device) * 1.0


In [ ]:
# 🔎 patchify 前向测试
with torch.no_grad():
    patches = net.patchify(concat_img)
    print(f"[patchify output shape] = {patches.shape}")


In [ ]:
# 🧠 注册所有 attention 层 hook，打印输出 shape
def make_hook(name):
    def hook(module, input, output):
        print(f"[Hook] {name} output shape: {output.shape}")
    return hook

for i, layer in enumerate(net.transformer.layers):
    if hasattr(layer, 'attn'):
        layer.attn.register_forward_hook(make_hook(f"Layer-{i}-Attn"))


In [ ]:
# 🚀 执行一次 model_forward（前向传播）
with torch.no_grad():
    out = net.model_forward(
        images=concat_img,
        lr_imgs=lr_img,
        sigmas=sigmas,
        input_ids=None, position_ids=None, attention_mask=None
    )
    print(f"[Final output shape] = {out.shape}")


In [ ]:
# 🏗️ 打印 Transformer 主体结构（层级）
print(net.transformer)


In [ ]:
# 🔍 选择第 0 层 Attention 结构展开看
print(net.transformer.layers[0].attn)
